In [ ]:
from dolfin import *
import matplotlib.pyplot as plt

def eps(u):
    #return sym(nabla_grad(u))
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

I = Identity(2)

u_degree = 3
p_degree = 2

n = 20
ratio = 4.0
mid = 2.0
mesh = RectangleMesh(Point(0.0), Point(ratio, 1.0), int(n*ratio), n)

fluidmarker = MeshFunction("size_t", mesh, mesh.topology().dim(), 0)
fluid = CompiledSubDomain("x[0] <= mid + DOLFIN_EPS", mid=mid)
fluid.mark(fluidmarker, 1)

porousmarker = MeshFunction("size_t", mesh, mesh.topology().dim(), 0)
porous = CompiledSubDomain("x[0] >= mid - DOLFIN_EPS", mid=mid)
porous.mark(porousmarker, 1)

bottom = CompiledSubDomain("on_boundary && near(x[1], 0.0)")
top = CompiledSubDomain("on_boundary && near(x[1], 1.0)")
left = CompiledSubDomain("on_boundary && near(x[0], 0.0)")
right = CompiledSubDomain("on_boundary && near(x[0], end)", end=ratio)
interface = CompiledSubDomain("near(x[0], mid, DOLFIN_EPS)", mid=mid)


boundary_int = MeshFunction("size_t", mesh, 1, 0)
marker_all = MeshFunction("size_t", mesh, 2, 0)
interface.mark(boundary_int, 1)

mesh_int = MeshView.create(boundary_int, 1)
mesh_all = MeshView.create(marker_all, 0)

porousmesh = MeshView.create(porousmarker, 1)
fluidmesh = MeshView.create(fluidmarker, 1)

boundary_intF = MeshFunction("size_t", fluidmesh, 1, 0)
boundary_intP = MeshFunction("size_t", porousmesh, 1, 0)

interface.mark(boundary_intF, 1)
interface.mark(boundary_intP, 1)

V = VectorElement("CG", mesh.ufl_cell(), u_degree)
W = FiniteElement("CG", mesh.ufl_cell(), p_degree)

# setup mixed space for porous region
VWWElement = MixedElement([V, W, W])
VWW = FunctionSpace(porousmesh, VWWElement)

# setup mixed space for fluid region
VWElement = MixedElement([V, W])
VW = FunctionSpace(fluidmesh, VWElement)

# setup mixed region function space
VWWVW = MixedFunctionSpace(VW, VWW)

# define trial functions
u_pF, d_pP_phi = TrialFunctions(VWWVW)
u, pF = split(u_pF)
d, pP, phi = split(d_pP_phi)

# define test functions
v_qF, w_qP_psi = TestFunctions(VWWVW)
v, qF = split(v_qF)
w, qP, psi = split(w_qP_psi)

# define constants

E = 1e5             # Young modulus
nu = 0.4             # Poisson ratio

A = 1.0
rho_s = Constant(1)
mu_s = Constant(E/(2.0*(1.0+nu)))
mu_s = Constant(1)

rho_f = Constant(1)
mu_f = Constant(1)

lmbda = Constant(nu*E/((1.0-2.0*nu)*(1.0+nu)))
lmbda = Constant(1)
alpha = Constant(1.0)
kappa = Constant(1)

#C = Constant(1.0)
gamma = Constant(0.3)

# define sources and forces

g = Constant([0.0, 0.0])
f = Constant([0.0, 0.0])


# define Measures
dxF = Measure("dx", domain=VWWVW.sub_space(0).mesh())
dxP = Measure("dx", domain=VWWVW.sub_space(1).mesh())

ds_Sig = Measure("dx", domain=mesh_int)
ds_SigF = Measure("ds", domain=fluidmesh, subdomain_data=boundary_intF)(1)
ds_SigP = Measure("ds", domain=porousmesh, subdomain_data=boundary_intP)(1)

n = Constant([1.0, 0.0])
t = Constant([0.0, 1.0])

# define forms
def a_F(u,v):
    return 2*mu_f*inner(eps(u), eps(v))*dxF + (gamma*mu_f/sqrt(kappa))*inner(u, t)*inner(v,t)*ds_SigF
    #return mu_f*inner((nabla_grad(u)), (nabla_grad(v)))*dxF + (gamma*mu_f/sqrt(kappa))*inner(u, t)*inner(v,t)*ds_SigF
            
def c_F(u,w,v):
    return rho_f * dot(dot(u, nabla_grad(w)), v)*dxF

def b_1_F(v, qF):
    return - qF*div(v)*dxF

def b_2_Sig(v, qP, ds):
    return qP*inner(v, n)*ds

def b_3_Sig(v, d, ds):
    return - ((gamma*mu_f/sqrt(kappa))*inner(v,t)*inner(d, t)*ds)

def b_4_Sig(w,qP, ds):
    return -qP * dot(w,n)*ds

def a_1_P(d, w):
    return 2.0*mu_s*inner(eps(d), eps(w))*dxP

def b_1_P(w, psi):
    return - psi*div(w)*dxP

def a_2_P(pP,qP):
    return (kappa/mu_f) *inner(grad(pP), grad(qP))*dxP

def b_2_P(psi, qP):
    return (alpha/lmbda)*psi*qP*dxP

def a_3_P(phi, psi):
    return (1.0/lmbda)*phi*psi*dxP

def F_F(v):
    return rho_f *dot(g, v)*dxF

def F_P(w):
    return rho_s*inner(f, w)*dxP

def G(qP):
    return rho_f*inner(g, grad(qP))*dxP  - rho_f*inner(g, n)*qP*ds_SigP 

# define system:

a1 = a_F(u,v) + b_1_F(v, pF) #+ c_F(u,u,v)

a2 = b_1_F(u, qF)

a3 = a_1_P(d,w) + b_1_P(w, phi) + b_4_Sig(w,pP, ds_SigP)

a4 = a_2_P(pP, qP)

a5 = b_1_P(d, psi) + b_2_P(psi, pP) - a_3_P(phi, psi)

coupling = True
if coupling:
    
    a1 += b_2_Sig(v, pP, ds_Sig)
    a3 += b_3_Sig(u, w, ds_Sig)
    a4 -= b_2_Sig(u, qP, ds_Sig)

a = a1 + a2 +  a3 + a4 + a5

L =  F_F(v) + F_P(w) + G(qP)

F = a -L

# define boundary conditions (fluid left and porous right)


#inflow = DirichletBC(VW.sub(0), Expression(["A*( -(2*x[1] - 1)*(2*x[1] - 1) + 1.0)", "0.0"],A=A, degree=2), left)
inflow = DirichletBC(VW.sub(0), Expression(("A*sin(x[1]*pi)", "0.0"),A=A, degree=2), left)
#inflow = DirichletBC(VW.sub(1), Constant(10.0), left)
outflow = DirichletBC(VW.sub(1), Constant(0.0), interface)
no_slip_top = DirichletBC(VW.sub(0), Constant([0.0, 0.0]), top)
no_slip_bottom = DirichletBC(VW.sub(0), Constant([0.0, 0.0]), bottom)

fluid_bcs = [no_slip_bottom, no_slip_top, inflow]


outflow_por = DirichletBC(VWW.sub(1), Constant(0.0), right)
inflow_por = DirichletBC(VWW.sub(1), Constant(10.0), interface)

fix_top = DirichletBC(VWW.sub(0), Constant([0.0, 0.0]), top)
fix_bottom = DirichletBC(VWW.sub(0), Constant([0.0, 0.0]), bottom)
fix_right = DirichletBC(VWW.sub(0), Constant([0.0, 0.0]), right)

#fix_interf = DirichletBC(VWW.sub(0), Constant([0.0, 0.0]), interface)


porous_bcs = [fix_bottom, fix_top,fix_right, outflow_por]#, inflow_por]
bcs = fluid_bcs + porous_bcs
sol = Function(VWWVW)
solve(a==L, sol,bcs=bcs,
      solver_parameters={"linear_solver":"direct"})


In [ ]:
u_pF, d_pP_phi = sol.split(True)
u, pF = u_pF.split(True)
d, pP, phi = d_pP_phi.split(True)
names = ["velocity u", "fluid pressure pF", "displacement d",
         "fluid pressure in porous domain pP", "total pressure phi",
        "darcy flow"]
funcs = [u, pF, d, pP, phi, project(-kappa*grad(pP))]
[f.rename(names[i], "") for i,f in enumerate(funcs)]

In [ ]:
# check interface conditions

# continuity of normal flux:

normal_flux_L = assemble(dot(u,n)*ds_Sig)
normal_flux_R = assemble((kappa/mu_f)* dot(-grad(pP),n)*ds_SigP)

print(f"normal fluxes: {normal_flux_L}, {normal_flux_R}")

In [ ]:
# momentum conservation_
mom_con_L0 = assemble(((2*mu_f*eps(u) - pF*I)*n)[0]*ds_SigF)
mom_con_L1 = assemble(((2*mu_f*eps(u) - pF*I)*n)[1]*ds_SigF)

mom_con_R0 = assemble(((2*mu_s*eps(d) - phi*I)* n)[0]*ds_SigP)
mom_con_R1 = assemble(((2*mu_s*eps(d) - phi*I)* n)[1]*ds_SigP)

print(f"mom con 0: {mom_con_L0}, {mom_con_R0}")
print(f"mom con 1: {mom_con_L1}, {mom_con_R1}")

In [ ]:
# fluid normal stresses
fns_L = assemble(- dot(n, (2*mu_f*eps(u) - pF*I)*n)*ds_Sig)
fns_R = assemble(pP*ds_Sig)
print(f"fluid normal stresses: {fns_L}, {fns_R}")

In [ ]:
for i, func in enumerate(funcs):
    plt.figure(figsize=(12,6))
    c = plot(func)
    plt.title(func.name())
    plt.colorbar(c)

In [ ]:
assemble(pP*ds_Sig)

In [ ]:
assemble(pP*ds_SigP)

In [ ]:
assemble(pF*ds_Sig)

In [ ]:
assemble(pF*ds_SigF)

In [ ]:
assemble(u[0]*ds_Sig)

In [ ]:
assemble(u[0]*ds_SigF)

In [ ]:
assemble(d[0]*ds_Sig)

In [ ]:
assemble(d[0]*ds_SigP)

In [ ]:
print(assemble(inner(grad(pP),n)*ds_Sig))
print(assemble(inner(grad(pP),n)*ds_SigP))
print(assemble(inner(grad(pP),n)*ds_SigF))


In [ ]:
print(assemble(pP*ds_Sig))
print(assemble(pP*ds_SigF))
print(assemble(pP*ds_SigP))

In [ ]:
print(assemble(pF*ds_Sig))
print(assemble(pF*ds_SigF))
print(assemble(pF*ds_SigP))

In [ ]:
print(assemble(inner(grad(pF),n)*ds_Sig))
print(assemble(inner(grad(pF),n)*ds_SigP))
print(assemble(inner(grad(pF),n)*ds_SigF))


In [ ]:
plt.figure(figsize=(8,8))
c = plot(grad(pP))
plt.colorbar(c)

In [ ]:
plot(mesh_int)

In [ ]:
mesh_int.num_vertices()